# 키워드 추출

## TF-IDF를 이용한 keyword 추출

In [ ]:
from konlpy.tag import Komoran

komoran = Komoran()
def komoran_tokenize(sents):
  i=0
  word=[]
  for sent in sents:
    i+=1
    words = komoran.pos(sent, join=True)
    words = [w.split("/")[0].lower() for w in words if ('/NN' in w or '/XR' in w or '/VA' in w) and len(w.split("/")[0]) > 1] #명사, 동사, 형용사   #or '/VV' in w
    word.append(" ".join(words))
  return word

In [ ]:
import konlpy

for z in range(0,1):
  feature0=perhotel[z]['sentence']
  nouns0=[]
  nouns1=[]
  twitter=Twitter()
  
  #twitter 형태소 사용
  for sentence in feature0:
    if sentence is not '':
      nouns0.append(' '.join([noun for noun in twitter.nouns(str(sentence))if noun not in stopwords and len(noun) > 1]))

  
  nouns0=komoran_tokenize(nouns0)

  cnt_vec = TfidfVectorizer()
  cnt_vec_mat = normalize(cnt_vec.fit_transform(nouns0).toarray().astype(float), axis=0)
  vocab0 = cnt_vec.vocabulary_
  words_graph0=np.dot(cnt_vec_mat.T, cnt_vec_mat)
  
  idx2word0={vocab0[word] : word for word in vocab0}

  A = words_graph0
  d=0.85
  matrix_size = A.shape[0]
  for id in range(matrix_size):
    A[id, id] = 0 # diagonal 부분을 0으로
    link_sum = np.sum(A[:,id]) # A[:, id] = A[:][id]
      
    if link_sum != 0:
        A[:, id] /= link_sum
    A[:, id] *= -d
    A[id, id] = 1
  B = (1-d) * np.ones((matrix_size, 1))
  ranks = np.linalg.solve(A, B)
  rank_idx={idx: r[0] for idx, r in enumerate(ranks)}

  sorted_rank_idx = sorted(rank_idx, key=lambda k: rank_idx[k], reverse=True)
  
  keywords = []
  index=[]
  for idx in sorted_rank_idx[:30]:
    index.append(idx)
  
  #index.sort()
  for idx in index:
    keywords.append(di.loc[di['0']==idx,'1'])
  
  keywords

/usr/local/lib/python3.7/dist-packages/konlpy/tag/_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


## 그룹핑하기

In [ ]:
from sklearn.mixture import GaussianMixture
from gensim.models import Word2Vec

feature0_0=perhotel[0]
feature0_0=feature0_0.drop(['level_0'],axis=1)
feature0_0=feature0_0.reset_index()

hotel0 = [
    '깔끔', '가성비', '깨끗', '강남', '가격', '위치', '직원', '이용', '친절', '먼지가', '시설', '만족',
    '출장', '생각보다', '없어서', '주변에', '보증금', '객실', '주차', '가까워서', '화장실', '레지던스',
    '오피스텔', '프런트', '청결', '접근성', '별로', '칫솔', '샴푸', '수건', '편의', '없는', '대비',
    '가깝고', '청소', '조용', '저렴', '느낌', '음식', '편하게', '방이', '취사', '아쉬', '어반', '불편',
    '서비스', '교통', '방도', '침구', '냄비', '엘리베이터', '가능', '어메니티'
] #각 호텔 키워드


nouns=[]
nouns=komoran_tokenize(feature0_0['sentence'])
embedding_model = Word2Vec(nouns, min_count=3,window=3,iter=20, size=100, sg=1)
embedding_model.wv.syn0.shape
word_vectors = embedding_model.wv.syn0


for z in range (0,len(embedding_model.wv.index2word)):
  if embedding_model.wv.index2word[z] in embedding_dict.keys():
      word_vectors[z]=embedding_dict[embedding_model.wv.index2word[z]]
  else:
    print("N")

word_vec=[]
name=[]
for z in range (0,len(embedding_model.wv.index2word)):
  if embedding_model.wv.index2word[z] in hotel0:
    word_vec.append(word_vectors[z])
    name.append(embedding_model.wv.index2word[z])

n=17 #로 미리 군집 개수 설정
num_clusters=16
kmeans_clustering = KMeans( n_clusters = num_clusters )#defaul
idx = kmeans_clustering.fit_predict(word_vectors) #각 단어가 어떤 군집에 속하는지 확인

idx = list(idx)
names = embedding_model.wv.index2word #학습된 단어들 unique
word_centroid_map = {names[i]: idx[i] for i in range(len(names))}

print("--------------------Hotel-------------------")
print(i)
answer=[]
for cluster in range(0,16):
  words = []
  #list(dict.values)는 value값이 리스트로 반환
  #list(dict)키 값만 리스트로 반환
  for i in range(0,len(list(word_centroid_map.values()))):
    if( list(word_centroid_map.values())[i] == cluster ):
      words.append(list(word_centroid_map.keys())[i])
  answer.append(words)
print(answer)
print("--------------------Next-------------------")

## 대표 뽑기

In [ ]:
nouns1=komoran_tokenize1(feature0_0['sentence'])
print(nouns1)
tfidfv = TfidfVectorizer().fit(nouns1)
tfidf=tfidfv.transform(nouns1).toarray()

dictionary = {} #대표 단어 key, 대표 단어가 속한 그룹 value
for i in range(len(hotel0_k)): 
  # hotel0_k는 단어 그룹
  # [['방문'],['객실', '침대', '발코니'],['접근성', '주차', '로비', '편의', '공용'], ['방음'], ['가격', '업그레이드'],['컨디션'], 
  # ['현대', '시티'], ['위치', '주변'],['이용', '서비스'], ['코로나'],['친절', '청결'], ['시설', '건물'], ['교통'], 
  # ['프런트', '체크인', '아웃렛'],  ['동대문'],  ['깨끗', '깔끔'], ['직원', '의사']]
  if len(hotel0_k[i])>=2: # 한 그룹의 단어 갯수가 2개 이상일 경우 대표 단어 추출 과정 필요 
    #한 그룹의 각 단어의 TF-IDF의 값과 Word2Vec에서 제공해주는 다른 단어와의 유사도 값을 합산하여 최고점의 단어를 뽑기 위해
    #한 그룹의 단어 수 만큼 배열을 만듦(배열 ar)
    #https://www.koreascience.or.kr/article/JAKO201833469090754.pdf (p28)
    ar=[0]*len(hotel0_k[i])
    #hotel24에 hotel0_k의 i번째 그룹의 단어가 포함되어 있는 문장수 체크(c2에 저장)
    for j in range(len(hotel0_k[i])):
      c2=0
      for c in feature0_0['sentence']: #hotel24의 sentence(feature0_0['sentence'])
        for c1 in hotel0_k[i]:
          if c1 in c:
            c2+=1
            break
      #hotel0_k에서 i번째 그룹의 j번째 단어가 vocabulary 몇 번째에 있는지 z에 저장
      z=tfidfv.vocabulary_[hotel0_k[i][j]]
      #hotel0_k에서 i번째 그룹의 j번째 단어의 TF-IDF의 값(t에 저장)
      t=0
      for j1 in range(len(tfidf)):
        t+=tfidf[j1][z]
      #t=t/len(tfidf)
      t=t/c2
      #hotel0_k에서 i번째 그룹의 j번째 단어가 i번째 그룹의 다른 단어와의 유사도 값 합산(t1에 저장)
      t1=0
      for j1 in range(len(hotel0_k[i])):
        if j1 != j:
          t1+=embedding_model.wv.similarity(w1=hotel0_k[i][j], w2=hotel0_k[i][j1])
      t1=t1/(len(hotel0_k[i])-1)
      #배열 ar의 j번째에 j번째의 TF-IDF값과 유사도 값을 합산한 값을 저장
      ar[j]=t+t1
    #가장 큰 합산 값을 대표 단어로 선정
    dictionary[hotel0_k[i][ar.index(max(ar))]]=hotel0_k[i] #dictionary[key]=value
  else: # 한 그룹의 단어 갯수가 1개면 그것이 대표 단어
    dictionary[hotel0_k[i][0]]=hotel0_k[i]
print(dictionary) #hotel24
#{'방문': ['방문'], '발코니': ['객실', '침대', '발코니'], '편의': ['접근성', '주차', '로비', '편의', '공용'], 
#'방음': ['방음'], '가격': ['가격', '업그레이드'], '컨디션': ['컨디션'], 
#'시티': ['현대', '시티'], '위치': ['위치', '주변'], '이용': ['이용', '서비스'], 
#'코로나': ['코로나'], '친절': ['친절', '청결'],'시설': ['시설', '건물'], '교통': ['교통'], 
#'프런트': ['프런트', '체크인', '아웃렛'], '동대문': ['동대문'], '깨끗': ['깨끗', '깔끔'], '직원': ['직원', '의사']}